In [2]:
import time
import numpy as np
from copy import deepcopy

In [15]:
class Sudoku:
    def __init__(self, board):
        self.board = board

        self.initial_domains = {}
        for i in range(9):
            for j in range(9):
                if self.board[i, j] == 0:
                    domain = set(range(1,10))
                    domain = domain - set(self.board[i, :]) ## rows
                    domain = domain - set(self.board[:, j]) # columns
                    box_i, box_j = i//3, j//3
                    box = board[3*box_i:3*box_i+3, 3*box_j:3*box_j+3]
                    domain = domain - set(box.reshape(-1))  # 3x3 boxes
                    self.initial_domains[(i, j)] = domain
        print('Initialized.')
        
        self.n_nodes_generated = 0

    def is_complete(self, board): # check if 0 exists
        return not (board==0).any()

    def check_consistency(self, board, i, j, value):
        if (board[i, :] == value).any():
            return False
        if (board[:, j] == value).any():
            return False
        box_i, box_j = i//3, j//3
        box = board[3*box_i:3*box_i+3, 3*box_j:3*box_j+3]
        if (box==value).any():
            return False
        return True

    def forward_checking(self, i, j, value, domains):
        box_i, box_j = i//3, j//3
        for (x, y) in domains:
            if x == i and y == j:
                continue
            if x == i or y == j or (i // 3 == x // 3 and j // 3 == y // 3):
                if value in domains[(x,y)]:
                    domains[(x,y)].remove(value)
                if len(domains[(x,y)]) == 0:
                    return None
        return domains

    def select_unassigned_variable_MRV(self, board, domains):
        # Select the cell with the smallest domain (minimum remaining values)
        empty_cells = list(zip(*np.nonzero(board==0)))
        return min(empty_cells, key=lambda cell: len(domains[cell])) if empty_cells else None

    def select_unassigned_variable_first(self, board, domains):
        # Select the cell with the smallest domain (minimum remaining values)
        empty_cells = list(zip(*np.nonzero(board==0)))
        return empty_cells[0]

    def order_domain_values(self, i, j, domains):
        # return domains[(i,j)]
        # Order values by the least constraining value heuristic
        values = list(domains[(i, j)])
        def count_constraints(value):
            count = 0
            for (x, y) in domains:
                if x == i or y == j or (i // 3 == x // 3 and j // 3 == y // 3):
                    if value in domains[(x, y)]:
                        count += 1
            return count
        return sorted(values, key=count_constraints)

    def backtrack(self, board, domains=None):
        if domains is None:
            domains = self.initial_domains

        if self.is_complete(board):
            return board

        # select a random variable for now
        variable = self.select_unassigned_variable_MRV(board, domains)
#         variable = self.select_unassigned_variable_first(board, domains)
        if variable is None:
            return None
        i, j = variable

        # order values
        ordered_values = self.order_domain_values(i, j, domains)
        for v in ordered_values:
            if not self.check_consistency(board, i, j, v):
                continue

            # make assignment
            new_board = board.copy()
            new_board[i, j] = v
            self.n_nodes_generated += 1
            # print(f'Assigning ({i}, {j}) = {v}')

            new_domains = deepcopy(domains)
            new_domains[(i, j)] = {v}
            new_domains = self.forward_checking(i, j, v, new_domains) # return None if any variable has an empty domain.
            if new_domains is None:
                continue

            result = self.backtrack(new_board, new_domains)
            if result is not None:
                return result
        return None

In [3]:
puzzle = '008300060,470000000,300007050,009800010,000000000,003020500,000700400,094008200,005100900'
board = np.array([[int(c) for c in l] for l in puzzle.split(',')])

In [9]:
board

array([[0, 0, 8, 3, 0, 0, 0, 6, 0],
       [4, 7, 0, 0, 0, 0, 0, 0, 0],
       [3, 0, 0, 0, 0, 7, 0, 5, 0],
       [0, 0, 9, 8, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 3, 0, 2, 0, 5, 0, 0],
       [0, 0, 0, 7, 0, 0, 4, 0, 0],
       [0, 9, 4, 0, 0, 8, 2, 0, 0],
       [0, 0, 5, 1, 0, 0, 9, 0, 0]])

In [12]:
puzzle = '008300060,470000000,300007050,009800010,000000000,003020500,000700400,094008200,005100900'
board = np.array([[int(c) for c in l] for l in puzzle.split(',')])

csp = Sudoku(board)

tic = time.time()
result = csp.backtrack(board)
print(f'Time used: {time.time()-tic:.1f}s')

if result is not None:
    print('Problem solved:')
    print(result)
else:
    print('No solution exists.')

print('Number of nodes generated:', csp.n_nodes_generated)

Initialized.
Time used: 85.4s
Problem solved:
[[9 5 8 3 1 4 7 6 2]
 [4 7 1 2 5 6 3 9 8]
 [3 2 6 9 8 7 1 5 4]
 [2 4 9 8 7 5 6 1 3]
 [5 6 7 4 3 1 8 2 9]
 [1 8 3 6 2 9 5 4 7]
 [6 1 2 7 9 3 4 8 5]
 [7 9 4 5 6 8 2 3 1]
 [8 3 5 1 4 2 9 7 6]]
Number of nodes generated: 297557


In [14]:
puzzle = '008300060,470000000,300007050,009800010,000000000,003020500,000700400,094008200,005100900'
board = np.array([[int(c) for c in l] for l in puzzle.split(',')])

csp = Sudoku(board)

tic = time.time()
result = csp.backtrack(board)
print(f'Time used: {time.time()-tic:.1f}s')

if result is not None:
    print('Problem solved:')
    print(result)
else:
    print('No solution exists.')

print('Number of nodes generated:', csp.n_nodes_generated)

Initialized.
Time used: 5.9s
Problem solved:
[[9 5 8 3 1 4 7 6 2]
 [4 7 1 2 5 6 3 9 8]
 [3 2 6 9 8 7 1 5 4]
 [2 4 9 8 7 5 6 1 3]
 [5 6 7 4 3 1 8 2 9]
 [1 8 3 6 2 9 5 4 7]
 [6 1 2 7 9 3 4 8 5]
 [7 9 4 5 6 8 2 3 1]
 [8 3 5 1 4 2 9 7 6]]
Number of nodes generated: 26233


In [16]:
puzzle = '008300060,470000000,300007050,009800010,000000000,003020500,000700400,094008200,005100900'
board = np.array([[int(c) for c in l] for l in puzzle.split(',')])

csp = Sudoku(board)

tic = time.time()
result = csp.backtrack(board)
print(f'Time used: {time.time()-tic:.1f}s')

if result is not None:
    print('Problem solved:')
    print(result)
else:
    print('No solution exists.')

print('Number of nodes generated:', csp.n_nodes_generated)

Initialized.
Time used: 0.3s
Problem solved:
[[9 5 8 3 1 4 7 6 2]
 [4 7 1 2 5 6 3 9 8]
 [3 2 6 9 8 7 1 5 4]
 [2 4 9 8 7 5 6 1 3]
 [5 6 7 4 3 1 8 2 9]
 [1 8 3 6 2 9 5 4 7]
 [6 1 2 7 9 3 4 8 5]
 [7 9 4 5 6 8 2 3 1]
 [8 3 5 1 4 2 9 7 6]]
Number of nodes generated: 1093
